In [1]:
import pandas as pd
import numpy as np
import pathlib
import json
import re

tables_num = ['I.','II.','III.','IV.','V.', 'VI.']

def all_null(row):
    if len(row.unique())==1:
        if np.isnan(row.unique()[0]):
            return 'drop'
    return 'keep'

def sections(x):
    if (type(x)!=float)&(type(x)!=int):
        for i in tables_num:
            #print x, i
            if x.split(' ')[0]==i:
                return i
    return np.nan

def check_empty(d):
    l_t = []
    if d.shape[0]==0:
        for i in range(d.shape[1]):
            l_t.append(np.nan)
        d.loc[0] = l_t
    return d

def sub_sections(x,subsets):
    if (type(x)!=float)&(type(x)!=int):
        for i in subsets:
            #print x, i
            if x.split(' ')[0]==i:
                return i
    return np.nan

def all_5_nan(row):
    row = row[1:]
    if len(row.unique())==1:
        return 'drop'
    else:
        return 'keep'

def put_nulls(df):
    for col in df:
        df[col]=df[col].str.replace(r"^(\[)(.*?)(\])$",'null')
    return df

In [2]:
def crea_json(ruta):
    df = pd.read_excel(ruta,index_col=0, header=None)
    df.reset_index(inplace=True, drop=True)
    
    df.dropna(how='all')
    df['act'] = df.apply(lambda row: all_null(row), axis=1)
    df = df[df['act']=='keep']
    
    #Para verificar el encabezado del dataframe para saber si de verdad es xlxs de patrimonio
    patron = re.compile('^(.*?)(patrimonial)(.*?)$')
    
    if(patron.search(df.iloc[0][0])):
        df.reset_index(inplace=True, drop=True)
        df['section'] = df[1].apply(lambda x: sections(x))  
        x='drop'
        x_l=[]
        for i in df['section'].tolist():
            if type(i)==str:
                x=i
            x_l.append(x)
        df['section']=x_l

        # SECCION I *****************************************************************************************
        sec_1 = df[df['section']=='I.']
        sec_1.reset_index(inplace=True, drop=True)
        sec_1_1 = sec_1.iloc[::,:2:].T
        sec_1_1.drop([0,1], axis=1, inplace=True)
        sec_1_1.columns = sec_1_1.loc[1]
        sec_1_1.drop(index=1, inplace=True)
        sec_1_1.reset_index(inplace=True, drop=True)
        sec_1_2 =sec_1.iloc[2::,5:7:].T
        sec_1_2.columns = sec_1_2.loc[6]
        sec_1_2.drop(index=6, inplace=True)
        sec_1_2.reset_index(inplace=True, drop=True)
        sec_1_final = pd.concat([sec_1_1,sec_1_2], axis=1)
        put_nulls(sec_1_final)

        # SECCION II ****************************************************************************************
        sec_2 = df[df['section']=='II.'].copy(deep=True)
        sec_2.reset_index(inplace=True, drop=True)
        sec_2.columns = sec_2.loc[2][:6].tolist() + sec_2.loc[3][6:].tolist()
        sec_2 = sec_2.iloc[5::,:-2:]
        col2=[u'Enliste e indique la relaci\xf3n entre el declarante con el c\xf3nyuge, concubina o concubinario y todo dependiente econ\xf3mico',
         u'Nacionalidad',
         u'\xbfHa desempe\xf1ado un cargo de gobierno \nen los \xfaltimos 5 a\xf1os?',
         u'Dependencia o entidad p\xfablica',
         u'Cargo, puesto o funci\xf3n',
         u'Periodo']
        sec_2 = sec_2[col2]
        sec_2 = check_empty(sec_2)
        sec_2.reset_index(inplace=True, drop=True)
        put_nulls(sec_2)

        # SECCION III ***************************************************************************************
        sec_3 = df[df['section']=='III.'].copy(deep=True)
        sec_3.reset_index(inplace=True, drop=True)
        subsets = ['1.','1.1']
        sec_3['section'] = sec_3[1].apply(lambda x: sub_sections(x,subsets))
        x='drop'
        x_l=[]
        for i in sec_3['section'].tolist():
            if type(i)==str:
                x=i
            x_l.append(x)
        sec_3['section']=x_l
        sec_3 = sec_3[sec_3['section']!='drop']

        sec_3_1 = sec_3[sec_3['section']=='1.']
        sec_3_1.reset_index(inplace=True, drop=True)
        sec_3_1 = sec_3_1.iloc[2::,::8].T.copy(deep=True)
        sec_3_1.columns = sec_3_1.loc[1]
        sec_3_1.drop(1, axis=0, inplace=True)
        col_3_1=[u'Remuneraci\xf3n neta anual del declarante por cargos p\xfablicos:',
        # u'Nota 1: Incluye sueldos, honorarios, compensaciones, bonos y otras prestaciones.',
        # u'Otros ingresos del declarante:',
         u'\u2013 Por actividades industrial, empresarial o comercial',
        # u'Nota 2: Se refiere a ingresos por actividades industriales, empresariales o comerciales en M\xe9xico y en el extranjero.',
         u'\u2013 Por actividad financiera',
        # u'Nota 3: Se refiere a ingresos por rendimiento de valores o contratos bancarios, plusval\xedas de participaci\xf3n accionaria e ingresos por pr\xe9stamos.           ',
         u'\u2013 Por servicios profesionales',
        # u'Nota 4: Los ingresos por servicios profesionales pueden incluir la participaci\xf3n en consejos, consultor\xedas o asesor\xedas de forma permanente u ocasional, en M\xe9xico y en el extranjero.',
         u'\u2013 Por otras actividades',
        # u'Nota 5: Se refiere a ingresos por arrendamientos, regal\xedas, sorteos, concursos, donaciones, entre otros.',
         u'Ingreso neto anual total del declarante:']
        sec_3_1 = sec_3_1[col_3_1]
        sec_3_1 = check_empty(sec_3_1)
        sec_3_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_3_1)

        sec_3_1_1 = sec_3[sec_3['section']=='1.1']
        sec_3_1_1.reset_index(inplace=True, drop=True)
        sec_3_1_1 = sec_3_1_1.iloc[::,:-3:8].T.copy(deep=True)
        sec_3_1_1.columns = sec_3_1_1.loc[1]
        col3_1_1=[#u'1.1 Ingreso anual neto del del C\xd3NYUGE, CONCUBINA O CONCUBINARIO Y/O DEPENDIENTES ECON\xd3MICOS entre el 1\xb0 de enero y el 31 de diciembre del a\xf1o inmediato anterior',
         #u'El ingreso neto anual del c\xf3nyuge, concubina o concubinario y dependientes econ\xf3micos se refiere a los ingresos netos \u2013despu\xe9s de impuestos\u2013 que tuvieron en el \xfaltimo a\xf1o fiscal concluido por actividades en cargos p\xfablicos, actividad industrial o comercial, actividad financiera, servicios profesionales \u2013incluyendo participaciones en consejos, consultor\xedas o asesor\xedas\u2013 as\xed como cualquier otro ingreso por actividades diversas. No es necesario desagregar por tipo de ingreso. Las cifras expresadas en esta secci\xf3n se declara en moneda nacional (MXN).',
         u'Ingreso anual neto del c\xf3nyuge, concubina o concubinario:',
         u'Ingreso anual neto de otros dependientes econ\xf3micos:',
         u'Ingreso neto anual total del c\xf3nyuge, concubina o concubinario y/o dependientes econ\xf3micos:']
        sec_3_1_1 = sec_3_1_1[col3_1_1]
        sec_3_1_1.drop(1, axis=0, inplace=True)
        sec_3_1_1 = check_empty(sec_3_1_1)
        sec_3_1_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_3_1_1)

        # SECCION IV ******************************************************************************
        sec_4 = df[df['section']=='IV.'].copy(deep=True)
        sec_4.reset_index(inplace=True, drop=True)
        subsets = ['1.', '1.1','2.', '2.1','3.', '3.1','4.','5.']
        sec_4['section'] = sec_4[1].apply(lambda x: sub_sections(x,subsets))
        x='drop'
        x_l=[]
        for i in sec_4['section'].tolist():
            if type(i)==str:
                x=i
            x_l.append(x)
        sec_4['section']=x_l
        sec_4 = sec_4[sec_4['section']!='drop']
        sec_4.reset_index(inplace=True, drop=True)
        # *********************************************************************************************
        sec_4_1 = sec_4[sec_4['section']=='1.']
        sec_4_1.reset_index(inplace=True, drop=True)
        sec_4_1.columns = sec_4_1.loc[2].tolist()[:1] + sec_4_1.loc[3].tolist()[1:3] + sec_4_1.loc[2].tolist()[3:]
        sec_4_1 = sec_4_1.iloc[4:-1:,:-2:]
        sec_4_1 = check_empty(sec_4_1)
        sec_4_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_1)
        # *********************************************************************************************
        sec_4_1_1 = sec_4[sec_4['section']=='1.1']
        sec_4_1_1.reset_index(inplace=True, drop=True)
        sec_4_1_1.columns = sec_4_1_1.loc[2]
        sec_4_1_1 = sec_4_1_1.iloc[4:-1:,:-5:]
        col4_1_1=[u'Tipo de bien declarado',
         u'Pa\xeds donde se encuentra el bien inmueble',
         u'Entidad federativa y municipio o delegaci\xf3n donde se encuentra el bien inmueble',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien inmueble',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien inmueble',
         u'Titular del bien inmueble declarado']
        sec_4_1_1 = sec_4_1_1[col4_1_1]
        sec_4_1_1 = check_empty(sec_4_1_1)
        sec_4_1_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_1_1)
        # *********************************************************************************************
        sec_4_2 = sec_4[sec_4['section']=='2.']
        sec_4_2.reset_index(inplace=True, drop=True)
        sec_4_2.columns = sec_4_2.loc[2]
        sec_4_2 = sec_4_2.iloc[4:-1:,:-2:]
        col4_2=[u'Tipo de veh\xedculo',
         u'Marca',
         u'Modelo',
         u'Pa\xeds donde est\xe1 registrado el veh\xedculo',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el veh\xedculo',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del veh\xedculo',
         u'Valor del veh\xedculo',
         u'Moneda',
         u'Titular del veh\xedculo']
        sec_4_2 = sec_4_2[col4_2]
        sec_4_2 = check_empty(sec_4_2)
        sec_4_2.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_2)
        # *********************************************************************************************
        sec_4_2_1 = sec_4[sec_4['section']=='2.1']
        sec_4_2_1.reset_index(inplace=True, drop=True)
        sec_4_2_1.columns = sec_4_2_1.loc[2]
        sec_4_2_1 = sec_4_2_1.iloc[3::,:-6:]
        col4_2_1=[u'Tipo de veh\xedculo',
         u'Pa\xeds donde est\xe1 registrado el veh\xedculo',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el veh\xedculo',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del veh\xedculo',
         u'Titular del veh\xedculo']
        sec_4_2_1 = sec_4_2_1[col4_2_1]
        sec_4_2_1 = check_empty(sec_4_2_1)
        sec_4_2_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_2_1)
        # *********************************************************************************************
        sec_4_3 = sec_4[sec_4['section']=='3.']
        sec_4_3.reset_index(inplace=True, drop=True)
        sec_4_3.columns = sec_4_3.loc[2]
        sec_4_3 = sec_4_3.iloc[4:-1:,:-4:]
        col4_3=[u'Tipo de bien mueble',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien mueble','nan',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien mueble',
         u'Valor estimado del bien mueble', 'nan',
         u'Moneda',
         u'Titular del bien mueble']
        if(len(sec_4_3.columns)==len(col4_3)):
            sec_4_3.columns = col4_3
        col4_3=[u'Tipo de bien mueble',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien mueble',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien mueble',
         u'Valor estimado del bien mueble',
         u'Moneda',
         u'Titular del bien mueble']
        sec_4_3 = sec_4_3[col4_3]
        sec_4_3 = check_empty(sec_4_3)
        sec_4_3.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_3)
        # *********************************************************************************************
        sec_4_3_1 = sec_4[sec_4['section']=='3.1']
        sec_4_3_1.reset_index(inplace=True, drop=True)
        sec_4_3_1.columns = sec_4_3_1.loc[2]
        sec_4_3_1 = sec_4_3_1.iloc[4:-1:,:-7:]
        col4_3_1=[u'Tipo de bien mueble',
         u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien mueble',
         u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien mueble',
         u'Titular del bien mueble']
        sec_4_3_1 = sec_4_3_1[col4_3_1]
        sec_4_3_1 = check_empty(sec_4_3_1)
        sec_4_3_1.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_3_1)
        # *********************************************************************************************
        sec_4_4 = sec_4[sec_4['section']=='4.']
        sec_4_4.reset_index(inplace=True, drop=True)
        sec_4_4.columns = sec_4_4.loc[2].tolist()[:5] + sec_4_4.loc[3].tolist()[5:8] + sec_4_4.loc[2].tolist()[8:]
        sec_4_4 = sec_4_4.iloc[4:-1:,:-2:]
        col4_4=[u'Tipo de inversi\xf3n',
         u'Pa\xeds donde est\xe1 constituida la entidad donde se encuentra la inversi\xf3n',
         u'Nombre o raz\xf3n social de la entidad',
         u'Menor o igual a $100,000.00',
         u'Entre $100,000.01 y $500,000.00',
         u'Mayor o igual a $500,000.01',
         u'Moneda de la inversi\xf3n',
         u'Titularidad de la inversi\xf3n']
        sec_4_4 = sec_4_4[col4_4]
        sec_4_4 = check_empty(sec_4_4)
        sec_4_4.reset_index(inplace=True, drop=True)
        put_nulls(sec_4_4)
        # SECCION V ******************************************************************************************
        sec_5 = df[df['section']=='V.']
        sec_5.reset_index(inplace=True, drop=True)
        sec_5.columns = sec_5.loc[2]
        sec_5 = sec_5.loc[3::,::].copy(deep=True)
        sec_5.reset_index(inplace=True, drop=True)
        sec_5 = sec_5.iloc[:-2:,:-6:]
        sec_5['tag'] = sec_5.apply(lambda row: all_5_nan(row), axis=1)
        sec_5 = sec_5[sec_5['tag'] =='keep'].copy(deep=True)
        sec_5.drop('tag', axis=1,inplace=True)
        col4_5=[u'No. de nota',
         u'Secci\xf3n a la que hace referencia',
         u'Subsecci\xf3n a la que hace referencia',
         u'Descripci\xf3n de la nota aclaratoria']
        sec_5 = sec_5[col4_5]
        sec_5 = check_empty(sec_5)
        sec_5.reset_index(inplace=True, drop=True)
        put_nulls(sec_5)
        
        table_list = {'sec_1':sec_1_final, 
        'sec_2':sec_2, #'sec_2_1': sec_2_1_1,'sec_2_2':sec_2_2, 'sec_2_2_1':sec_2_2_1,'sec_2_3':sec_2_3,'sec_2_3_1':sec_2_3_1,'sec_2_4':sec_2_4,'sec_2_5':sec_2_5, 
        'sec_3_1':sec_3_1, 'sec_3_1_1':sec_3_1_1,# 'sec_3_3':sec_3_3, 
        'sec_4_1':sec_4_1, 'sec_4_1_1':sec_4_1_1, 'sec_4_2':sec_4_2, 'sec_4_2_1':sec_4_2_1, 'sec_4_3':sec_4_3, 'sec_4_3_1':sec_4_3_1,  'sec_4_4':sec_4_4, #'sec_4_5':sec_4_5,
        'sec_5':sec_5}

        the_json = {'persona':{sec_1_final['Nombre:'].values[0] + '_' + str(df[10][0]).split(' ')[0]:
             {'sec_1':sec_1_final.to_json(orient='records'),
              'sec_2':sec_2.to_json(orient='records'),
              'sec_3':{
                'sec_3_1':sec_3_1.to_json(orient='records'), 'sec_3_1_1':sec_3_1_1.to_json(orient='records'),
              },
              'sec_4':{
                  'sec_4_1':sec_4_1.to_json(orient='records'), 'sec_4_1_1':sec_4_1_1.to_json(orient='records'), 'sec_4_2':sec_4_2.to_json(orient='records'), 
                  'sec_4_2_1':sec_4_2_1.to_json(orient='records'), 'sec_4_3':sec_4_3.to_json(orient='records'), 'sec_4_3_1':sec_4_3_1.to_json(orient='records'),  'sec_4_4':sec_4_4.to_json(orient='records')
              },
              'sec_5':sec_5.to_json(orient='records'),
             }
            }
        }

        with open('Output/Patrimonio/jsons/patrimonio_'+sec_1_final['Nombre:'].values[0].replace(' ','')+'.json', 'w') as fp:
            json.dump(the_json, fp)
    else:
        "El excel dice que es patrimonial pero es de intereses"

In [3]:
def find_DP_xlsx(cadena):
    #Para verificar si el archivo .xlsx se trata de un excel de patrimonio
    patron = re.compile('^(.*?)(_DP| DP|DP |declaracion_patrimonial|DP_)(.*?)(\.xlsx)$')
    if(patron.search(cadena)):
        print(cadena)
        crea_json(cadena)

In [4]:
directorio_raiz =  pathlib.Path('Data/3de3_CDMX/')

for archivos_1 in directorio_raiz.iterdir():
    directorio_2=str(archivos_1)
    directorio_2=pathlib.Path(directorio_2)
    for archivos_2 in directorio_2.iterdir():
        directorio_3=str(archivos_2)
        directorio_3_aux=str(directorio_3)
        if(directorio_3_aux.count('pdf')==0 and directorio_3_aux.count('xlsx')==0):
            directorio_3 = pathlib.Path(directorio_3)
            for archivos_3 in directorio_3.iterdir():
                excel=str(archivos_3)
                if(excel.count('xls')==1):
                    find_DP_xlsx(str(archivos_3))
        else:
            find_DP_xlsx(str(directorio_3_aux))

Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Beatriz_Adriana_Olivares_Pinal/01 DP BAOP.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Rafael_Perez_Laurrabaquio/01_DP_RPL.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/02_Jaime_Ruben_Morales_Beltran/01_DP_JRMB.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/02_Araceli_Damian_Gonzalez/01_DP_ADG.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/01_Almudena_Ocejo_Rojo/01_DP_AOR.xlsx
Data/3de3_CDMX/Secretaria_de_Inclusion_y_Bienestar_Social/03_Rafael_Vargas_Morales/01_DP_RMV.xlsx
Data/3de3_CDMX/Procuraduria_de_Justicia/02_Rodrigo_de_la_Riva/01_DP_RRR.xlsx
Data/3de3_CDMX/Coordinacion_General_de_Comunicacion_Social/01_Cesar_Ivan_Escalante_Ruiz/01_DP_CIER.xlsx
Data/3de3_CDMX/Consejo_Ciudadano/declaracion_patrimonial (1).xlsx
Data/3de3_CDMX/Secretaria_para_la_Autonomia_de_la_Mujer/03_Maria_Luisa_Leticia_Silva_Canaan/01_DP_MLLSC.xlsx
Data/3de3_CDMX/Secretaria_para_la_A

Data/3de3_CDMX/Secretaria_de_Movilidad/02_Maria_de_los_Angeles_Munoz/01_DP_MAM.xlsx
Data/3de3_CDMX/Secretaria_de_Desarrollo_Economico/02_Roxana_Aguirre_Elizondo/01_DP_RAE.xlsx
Data/3de3_CDMX/Secretaria_de_Desarrollo_Economico/03_Jose_Alberto_Valdes_Palacios/01_DP_JAVP.xlsx
Data/3de3_CDMX/Secretaria_de_Desarrollo_Economico/01_Jose_Luis_Beato_Gonzalez/01_DP_JLBG.xlsx
Data/3de3_CDMX/Comision_de_Reconstruccion/01_Cesar_Arnulfo_Cravioto_Romero/01_DP_CARC.xlsx
Data/3de3_CDMX/Comision_de_Reconstruccion/03_Mario_Isaac_Valderrama_Gonzalez/01_DP_MIGV.xlsx
Data/3de3_CDMX/Centro_Comando_Control_Computo_Comunicaciones_Contacto_Ciudadano_C5/03_Juan_Antonio_Suarez_Sanchez/01_DP_JASS.xlsx
Data/3de3_CDMX/Centro_Comando_Control_Computo_Comunicaciones_Contacto_Ciudadano_C5/03_Edith_Palomera/01_DP_EPM.xlsx
Data/3de3_CDMX/Centro_Comando_Control_Computo_Comunicaciones_Contacto_Ciudadano_C5/03_Francisco_Saldaдa/01_DP_FSL.xlsx
Data/3de3_CDMX/Centro_Comando_Control_Computo_Comunicaciones_Contacto_Ciudadano_C5/

In [5]:
from pprint import pprint

json_completo=[]
directorio_jsons =  pathlib.Path('Output/Patrimonio/jsons/')

for archivo in directorio_jsons.iterdir():
    with open(str(archivo)) as f:
        data = json.load(f)
        json_completo.append(data)
#pprint(json_completo)

with open('Output/Patrimonio/json_completo/json_completo_patrimonio.json', 'w') as fp:
    json.dump(json_completo, fp)